> ライブラリ&データ読み込み

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.ml.feature import OneHotEncoder

spark = SparkSession.builder \
        .appName("Data_wrangling") \
        .getOrCreate()

df_customer = spark.read.format("csv") \
    .option("inferSchema", "True") \
    .option("header", "True") \
    .option("sep", ",") \
    .load("./data/customer.csv")
df_category = spark.read.format("csv") \
    .option("inferSchema", "True") \
    .option("header", "True") \
    .option("sep", ",") \
    .load("./data/category.csv")
df_product = spark.read.format("csv") \
    .option("inferSchema", "True") \
    .option("header", "True") \
    .option("sep", ",") \
    .load("./data/product.csv")
df_receipt = spark.read.format("csv") \
    .option("inferSchema", "True") \
    .option("header", "True") \
    .option("sep", ",") \
    .load("./data/receipt.csv")
df_store = spark.read.format("csv") \
    .option("inferSchema", "True") \
    .option("header", "True") \
    .option("sep", ",") \
    .load("./data/store.csv")
df_geocode = spark.read.format("csv") \
    .option("inferSchema", "True") \
    .option("header", "True") \
    .option("sep", ",") \
    .load("./data/geocode.csv")

 > P-001: レシート明細データ（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [13]:
df_receipt.show(10)

+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
|sales_ymd|sales_epoch|store_cd|receipt_no|receipt_sub_no|   customer_id|product_cd|quantity|amount|
+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
| 20181103| 1541203200|  S14006|       112|             1|CS006214000001|P070305012|       1|   158|
| 20181118| 1542499200|  S13008|      1132|             2|CS008415000097|P070701017|       1|    81|
| 20170712| 1499817600|  S14028|      1102|             1|CS028414000014|P060101005|       1|   170|
| 20190205| 1549324800|  S14042|      1132|             1|ZZ000000000000|P050301001|       1|    25|
| 20180821| 1534809600|  S14025|      1102|             2|CS025415000050|P060102007|       1|    90|
| 20190605| 1559692800|  S13003|      1112|             1|CS003515000195|P050102002|       1|   138|
| 20181205| 1543968000|  S14024|      1102|             2|CS024514000042|P080101005|       

 > P-002: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。

In [14]:
df_receipt.select("sales_ymd", "customer_id", "product_cd", "amount").show(10)

+---------+--------------+----------+------+
|sales_ymd|   customer_id|product_cd|amount|
+---------+--------------+----------+------+
| 20181103|CS006214000001|P070305012|   158|
| 20181118|CS008415000097|P070701017|    81|
| 20170712|CS028414000014|P060101005|   170|
| 20190205|ZZ000000000000|P050301001|    25|
| 20180821|CS025415000050|P060102007|    90|
| 20190605|CS003515000195|P050102002|   138|
| 20181205|CS024514000042|P080101005|    30|
| 20190922|CS040415000178|P070501004|   128|
| 20170504|ZZ000000000000|P071302010|   770|
| 20191010|CS027514000015|P071101003|   680|
+---------+--------------+----------+------+
only showing top 10 rows



 > P-003: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。ただし、sales_ymdをsales_dateに項目名を変更して抽出すること。

In [15]:
df_receipt.select("sales_ymd", "customer_id", "product_cd", "amount").withColumnRenamed("sales_ymd", "sales_date").show(10)

+----------+--------------+----------+------+
|sales_date|   customer_id|product_cd|amount|
+----------+--------------+----------+------+
|  20181103|CS006214000001|P070305012|   158|
|  20181118|CS008415000097|P070701017|    81|
|  20170712|CS028414000014|P060101005|   170|
|  20190205|ZZ000000000000|P050301001|    25|
|  20180821|CS025415000050|P060102007|    90|
|  20190605|CS003515000195|P050102002|   138|
|  20181205|CS024514000042|P080101005|    30|
|  20190922|CS040415000178|P070501004|   128|
|  20170504|ZZ000000000000|P071302010|   770|
|  20191010|CS027514000015|P071101003|   680|
+----------+--------------+----------+------+
only showing top 10 rows



 > P-004: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
 > - 顧客ID（customer_id）が"CS018205000001"

In [16]:
df_receipt.select("sales_ymd", "customer_id", "product_cd", "amount") \
    .filter(F.col("customer_id") == "CS018205000001") \
    .show()

+---------+--------------+----------+------+
|sales_ymd|   customer_id|product_cd|amount|
+---------+--------------+----------+------+
| 20180911|CS018205000001|P071401012|  2200|
| 20180414|CS018205000001|P060104007|   600|
| 20170614|CS018205000001|P050206001|   990|
| 20170614|CS018205000001|P060702015|   108|
| 20190216|CS018205000001|P071005024|   102|
| 20180414|CS018205000001|P071101002|   278|
| 20190226|CS018205000001|P070902035|   168|
| 20190924|CS018205000001|P060805001|   495|
| 20190226|CS018205000001|P071401020|  2200|
| 20180911|CS018205000001|P071401005|  1100|
| 20190216|CS018205000001|P040101002|   218|
| 20190924|CS018205000001|P091503001|   280|
+---------+--------------+----------+------+



 > P-005: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
 > - 顧客ID（customer_id）が"CS018205000001"
 > - 売上金額（amount）が1,000以上

In [17]:
df_receipt.select("sales_ymd", "customer_id", "product_cd", "amount") \
    .filter((F.col("customer_id") == "CS018205000001") & (F.col("amount")>=1000)) \
    .show()

+---------+--------------+----------+------+
|sales_ymd|   customer_id|product_cd|amount|
+---------+--------------+----------+------+
| 20180911|CS018205000001|P071401012|  2200|
| 20190226|CS018205000001|P071401020|  2200|
| 20180911|CS018205000001|P071401005|  1100|
+---------+--------------+----------+------+



 > P-006: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
 > - 顧客ID（customer_id）が"CS018205000001"
 > - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [18]:
df_receipt.select("sales_ymd", "customer_id", "product_cd", "quantity", "amount") \
    .filter(F.col("customer_id") == "CS018205000001") \
    .filter((F.col("amount")>=1000) | (F.col("quantity") >= 5)) \
    .show()

+---------+--------------+----------+--------+------+
|sales_ymd|   customer_id|product_cd|quantity|amount|
+---------+--------------+----------+--------+------+
| 20180911|CS018205000001|P071401012|       1|  2200|
| 20180414|CS018205000001|P060104007|       6|   600|
| 20170614|CS018205000001|P050206001|       5|   990|
| 20190226|CS018205000001|P071401020|       1|  2200|
| 20180911|CS018205000001|P071401005|       1|  1100|
+---------+--------------+----------+--------+------+



 > P-007: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
 > - 顧客ID（customer_id）が"CS018205000001"
 > - 売上金額（amount）が1,000以上2,000以下

In [19]:
df_receipt.select("sales_ymd", "customer_id", "product_cd", "amount") \
    .filter(F.col("customer_id") == "CS018205000001") \
    .filter((F.col("amount")>=1000) & (F.col("amount") <= 2000)) \
    .show()

+---------+--------------+----------+------+
|sales_ymd|   customer_id|product_cd|amount|
+---------+--------------+----------+------+
| 20180911|CS018205000001|P071401005|  1100|
+---------+--------------+----------+------+



 > P-008: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
 > - 顧客ID（customer_id）が"CS018205000001"
 > - 商品コード（product_cd）が"P071401019"以外

In [20]:
df_receipt.select("sales_ymd", "customer_id", "product_cd", "amount") \
    .filter((F.col("customer_id") == "CS018205000001") & (F.col("product_cd") != "P071401019")) \
    .show()

+---------+--------------+----------+------+
|sales_ymd|   customer_id|product_cd|amount|
+---------+--------------+----------+------+
| 20180911|CS018205000001|P071401012|  2200|
| 20180414|CS018205000001|P060104007|   600|
| 20170614|CS018205000001|P050206001|   990|
| 20170614|CS018205000001|P060702015|   108|
| 20190216|CS018205000001|P071005024|   102|
| 20180414|CS018205000001|P071101002|   278|
| 20190226|CS018205000001|P070902035|   168|
| 20190924|CS018205000001|P060805001|   495|
| 20190226|CS018205000001|P071401020|  2200|
| 20180911|CS018205000001|P071401005|  1100|
| 20190216|CS018205000001|P040101002|   218|
| 20190924|CS018205000001|P091503001|   280|
+---------+--------------+----------+------+



 > P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。
 >
 > `df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [21]:
df_store.filter((F.col("prefecture_cd") != "13") & (F.col("floor_area") <= 900)) \
    .show()

+--------+----------+-------------+----------+----------------------------------+--------------------------------------+------------+---------+--------+----------+
|store_cd|store_name|prefecture_cd|prefecture|                           address|                          address_kana|      tel_no|longitude|latitude|floor_area|
+--------+----------+-------------+----------+----------------------------------+--------------------------------------+------------+---------+--------+----------+
|  S14046|  北山田店|           14|  神奈川県|  神奈川県横浜市都筑区北山田一丁目| カナガワケンヨコハマシツヅキクキタ...|045-123-4049| 139.5916|35.56189|     831.0|
|  S14011|日吉本町店|           14|  神奈川県|神奈川県横浜市港北区日吉本町四丁目| カナガワケンヨコハマシコウホククヒ...|045-123-4033| 139.6316|35.54655|     890.0|
|  S12013|  習志野店|           12|    千葉県|          千葉県習志野市芝園一丁目|チバケンナラシノシシバゾノイッチョウメ|047-123-4002|  140.022|35.66122|     808.0|
+--------+----------+-------------+----------+----------------------------------+--------------------------------------+------------+-----

 > P-010: 店舗データ（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件表示せよ。

In [22]:
df_store.filter(F.col("store_cd").rlike("^S14")) \
    .show(10)

+--------+------------+-------------+----------+--------------------------------------+----------------------------------------+------------+---------+--------+----------+
|store_cd|  store_name|prefecture_cd|prefecture|                               address|                            address_kana|      tel_no|longitude|latitude|floor_area|
+--------+------------+-------------+----------+--------------------------------------+----------------------------------------+------------+---------+--------+----------+
|  S14010|      菊名店|           14|  神奈川県|        神奈川県横浜市港北区菊名一丁目|   カナガワケンヨコハマシコウホククキ...|045-123-4032| 139.6326|35.50049|    1732.0|
|  S14033|    阿久和店|           14|  神奈川県|    神奈川県横浜市瀬谷区阿久和西一丁目|   カナガワケンヨコハマシセヤクアクワ...|045-123-4043| 139.4961|35.45918|    1495.0|
|  S14036|相模原中央店|           14|  神奈川県|            神奈川県相模原市中央二丁目|   カナガワケンサガミハラシチュウオウ...|042-123-4045| 139.3716|35.57327|    1679.0|
|  S14040|    長津田店|           14|  神奈川県|神奈川県横浜市緑区長津田みなみ台五丁目|   カナガワケンヨコハマシミドリクナガ...|045-1

 > P-011: 顧客データ（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件表示せよ。

In [23]:
df_customer.filter(F.col("customer_id").rlike("1$")) \
    .show(10)

+--------------+-------------+---------+------+----------+---+---------+---------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                          address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+---------------------------------+--------------------+----------------+------------+
|CS037613000071|    六角 雅彦|        9|  不明|1952-04-01| 66| 136-0076|       東京都江東区南砂**********|              S13037|        20150414|0-00000000-0|
|CS028811000001|  堀井 かおり|        1|  女性|1933-03-27| 86| 245-0016| 神奈川県横浜市泉区和泉町*****...|              S14028|        20160115|0-00000000-0|
|CS040412000191|    川井 郁恵|        1|  女性|1977-01-05| 42| 226-0021|神奈川県横浜市緑区北八朔町****...|              S14040|        20151101|1-20091025-4|
|CS028314000011|  小菅 あおい|        1|  女性|1983-11-26| 35| 246-0038| 神奈川県横浜市瀬谷区宮沢*****...|              S14

 > P-012: 店舗データ（df_store）から、住所 (address) に"横浜市"が含まれるものだけ全項目表示せよ。

In [24]:
df_store.filter(F.col("address").contains("横浜市")) \
    .show()

+--------+----------+-------------+----------+--------------------------------------+----------------------------------------+------------+---------+--------+----------+
|store_cd|store_name|prefecture_cd|prefecture|                               address|                            address_kana|      tel_no|longitude|latitude|floor_area|
+--------+----------+-------------+----------+--------------------------------------+----------------------------------------+------------+---------+--------+----------+
|  S14010|    菊名店|           14|  神奈川県|        神奈川県横浜市港北区菊名一丁目|   カナガワケンヨコハマシコウホククキ...|045-123-4032| 139.6326|35.50049|    1732.0|
|  S14033|  阿久和店|           14|  神奈川県|    神奈川県横浜市瀬谷区阿久和西一丁目|   カナガワケンヨコハマシセヤクアクワ...|045-123-4043| 139.4961|35.45918|    1495.0|
|  S14040|  長津田店|           14|  神奈川県|神奈川県横浜市緑区長津田みなみ台五丁目|   カナガワケンヨコハマシミドリクナガ...|045-123-4046| 139.4994|35.52398|    1548.0|
|  S14050|阿久和西店|           14|  神奈川県|    神奈川県横浜市瀬谷区阿久和西一丁目|   カナガワケンヨコハマシセヤクアクワ...|045-123-4053| 139.4961

 > P-013: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件表示せよ。

In [25]:
df_customer.filter(F.col("status_cd").rlike("^[A-F]")) \
    .show(10)

+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                           address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|CS031415000172|宇多田 貴美子|        1|  女性|1976-10-04| 42| 151-0053|      東京都渋谷区代々木**********|              S13031|        20150529|D-20100325-C|
|CS015414000103|    奥野 陽子|        1|  女性|1977-08-09| 41| 136-0073|        東京都江東区北砂**********|              S13015|        20150722|B-20100609-B|
|CS011215000048|    芦田 沙耶|        1|  女性|1992-02-01| 27| 223-0062|神奈川県横浜市港北区日吉本町***...|              S14011|        20150228|C-20100421-9|
|CS029415000023|    梅田 里穂|        1|  女性|1976-01-17| 43| 279-0043|      千葉県浦安市富士見**********|     

 > P-014: 顧客データ（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [26]:
df_customer.filter(F.col("status_cd").rlike("[1-9]$")) \
    .show(10)

+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                           address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|CS001215000145|    田崎 美紀|        1|  女性|1995-03-29| 24| 144-0055|      東京都大田区仲六郷**********|              S13001|        20170605|6-20090929-2|
|CS033513000180|      安斎 遥|        1|  女性|1962-07-11| 56| 241-0823|  神奈川県横浜市旭区善部町*****...|              S14033|        20150728|6-20080506-5|
|CS011215000048|    芦田 沙耶|        1|  女性|1992-02-01| 27| 223-0062|神奈川県横浜市港北区日吉本町***...|              S14011|        20150228|C-20100421-9|
|CS040412000191|    川井 郁恵|        1|  女性|1977-01-05| 42| 226-0021| 神奈川県横浜市緑区北八朔町****...|          

 > P-015: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [27]:
df_customer.filter((F.col("status_cd").rlike("^[A-F]")) & (F.col("status_cd").rlike("[1-9]$"))) \
    .show(10)

+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                           address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+----------------------------------+--------------------+----------------+------------+
|CS011215000048|    芦田 沙耶|        1|  女性|1992-02-01| 27| 223-0062|神奈川県横浜市港北区日吉本町***...|              S14011|        20150228|C-20100421-9|
|CS022513000105|  島村 貴美子|        1|  女性|1962-03-12| 57| 249-0002|    神奈川県逗子市山の根**********|              S14022|        20150320|A-20091115-7|
|CS001515000096|    水野 陽子|        9|  不明|1960-11-29| 58| 144-0053|    東京都大田区蒲田本町**********|              S13001|        20150614|A-20100724-7|
|CS013615000053|    西脇 季衣|        1|  女性|1953-10-18| 65| 261-0026|  千葉県千葉市美浜区幕張西*****...|          

 > P-016: 店舗データ（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [28]:
df_store.filter(F.col("tel_no").rlike("[0-9]{3}-[0-9]{3}-[0-9]{4}")) \
    .show()

+--------+------------+-------------+----------+--------------------------------------+----------------------------------------+------------+---------+--------+----------+
|store_cd|  store_name|prefecture_cd|prefecture|                               address|                            address_kana|      tel_no|longitude|latitude|floor_area|
+--------+------------+-------------+----------+--------------------------------------+----------------------------------------+------------+---------+--------+----------+
|  S12014|    千草台店|           12|    千葉県|        千葉県千葉市稲毛区千草台一丁目|   チバケンチバシイナゲクチグサダイイ...|043-123-4003|  140.118|35.63559|    1698.0|
|  S13002|    国分寺店|           13|    東京都|              東京都国分寺市本多二丁目|トウキョウトコクブンジシホンダニチョウメ|042-123-4008| 139.4802|35.70566|    1735.0|
|  S14010|      菊名店|           14|  神奈川県|        神奈川県横浜市港北区菊名一丁目|   カナガワケンヨコハマシコウホククキ...|045-123-4032| 139.6326|35.50049|    1732.0|
|  S14033|    阿久和店|           14|  神奈川県|    神奈川県横浜市瀬谷区阿久和西一丁目|   カナガワケンヨコハマシセヤクアクワ...

 > P-017: 顧客データ（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭から全項目を10件表示せよ。

In [29]:
df_customer.orderBy(F.asc("birth_day")) \
    .show(10)

+--------------+-------------+---------+------+----------+---+---------+--------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                         address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+--------------------------------+--------------------+----------------+------------+
|CS003813000014|  村山 菜々美|        1|  女性|1928-11-26| 90| 182-0007|    東京都調布市菊野台**********|              S13003|        20160214|0-00000000-0|
|CS026813000004|    吉村 朝陽|        1|  女性|1928-12-14| 90| 251-0043| 神奈川県藤沢市辻堂元町******...|              S14026|        20150723|0-00000000-0|
|CS018811000003|    熊沢 美里|        1|  女性|1929-01-07| 90| 204-0004|      東京都清瀬市野塩**********|              S13018|        20150403|0-00000000-0|
|CS027803000004|    内村 拓郎|        0|  男性|1929-01-12| 90| 251-0031|神奈川県藤沢市鵠沼藤が谷*****...|              S140

 > P-018: 顧客データ（df_customer）を生年月日（birth_day）で若い順にソートし、先頭から全項目を10件表示せよ。

In [30]:
df_customer.orderBy(F.desc("birth_day")) \
    .show(10)

+--------------+-------------+---------+------+----------+---+---------+---------------------------------+--------------------+----------------+------------+
|   customer_id|customer_name|gender_cd|gender| birth_day|age|postal_cd|                          address|application_store_cd|application_date|   status_cd|
+--------------+-------------+---------+------+----------+---+---------+---------------------------------+--------------------+----------------+------------+
|CS035114000004|    大村 美里|        1|  女性|2007-11-25| 11| 156-0053|       東京都世田谷区桜**********|              S13035|        20150619|6-20091205-6|
|CS022103000002|  福山 はじめ|        9|  不明|2007-10-02| 11| 249-0006|     神奈川県逗子市逗子**********|              S14022|        20160909|0-00000000-0|
|CS002113000009|  柴田 真悠子|        1|  女性|2007-09-17| 11| 184-0014|  東京都小金井市貫井南町******...|              S13002|        20160304|0-00000000-0|
|CS004115000014|    松井 京子|        1|  女性|2007-08-09| 11| 165-0031|     東京都中野区上鷺宮**********|         

 > P-019: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [31]:
window = Window.partitionBy("amount").orderBy(F.desc("amount"))
df_receipt.select("customer_id", "amount") \
    .withColumn("rank", F.rank().over(window)) \
    .show(10)

+--------------+------+----+
|   customer_id|amount|rank|
+--------------+------+----+
|CS017415000158|    10|   1|
|CS026414000060|    10|   1|
|CS027513000078|    10|   1|
|ZZ000000000000|    10|   1|
|ZZ000000000000|    10|   1|
|CS006415000186|    10|   1|
|CS018414000049|    10|   1|
|CS025415000056|    10|   1|
|CS027414000084|    10|   1|
|CS028215000024|    10|   1|
+--------------+------+----+
only showing top 10 rows



 > P-020: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [32]:
window = Window.partitionBy("amount").orderBy(F.desc("amount"))
df_receipt.select("customer_id", "amount") \
    .withColumn("rank", F.row_number().over(window)) \
    .show(10)

+--------------+------+----+
|   customer_id|amount|rank|
+--------------+------+----+
|CS017415000158|    10|   1|
|CS026414000060|    10|   2|
|CS027513000078|    10|   3|
|ZZ000000000000|    10|   4|
|ZZ000000000000|    10|   5|
|CS006415000186|    10|   6|
|CS018414000049|    10|   7|
|CS025415000056|    10|   8|
|CS027414000084|    10|   9|
|CS028215000024|    10|  10|
+--------------+------+----+
only showing top 10 rows



 > P-021: レシート明細データ（df_receipt）に対し、件数をカウントせよ。

In [33]:
df_receipt.count()

104681

 > P-022: レシート明細データ（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [34]:
df_receipt.select("customer_id") \
    .distinct() \
    .count()

8307

 > P-023: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [35]:
df_receipt.groupBy("store_cd") \
    .agg({"amount":"sum", "quantity":"sum"}) \
    .show()

+--------+-----------+-------------+
|store_cd|sum(amount)|sum(quantity)|
+--------+-----------+-------------+
|  S14024|     736323|         2417|
|  S14045|     458484|         1398|
|  S13020|     796383|         2383|
|  S13038|     708884|         2337|
|  S14047|     338329|         1041|
|  S12014|     725167|         2358|
|  S13037|     693087|         2344|
|  S13051|     107452|          354|
|  S12013|     787513|         2425|
|  S14046|     412646|         1354|
|  S14011|     805724|         2434|
|  S13016|     793773|         2432|
|  S13035|     715869|         2219|
|  S14006|     712839|         2284|
|  S14027|     714550|         2303|
|  S13008|     809288|         2491|
|  S14025|     755581|         2394|
|  S13031|     705968|         2336|
|  S14036|     203694|          635|
|  S14040|     701858|         2233|
+--------+-----------+-------------+
only showing top 20 rows



 > P-024: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）を求め、10件表示せよ。

In [36]:
df_receipt.groupBy("customer_id") \
    .agg({"sales_ymd": "max"}) \
    .show(10)

+--------------+--------------+
|   customer_id|max(sales_ymd)|
+--------------+--------------+
|CS024415000195|      20190907|
|CS011515000044|      20181226|
|CS017415000245|      20190615|
|CS011512000113|      20190829|
|CS025414000093|      20191016|
|CS010605000007|      20181009|
|CS011415000097|      20180909|
|CS018515000090|      20190717|
|CS015415000222|      20190928|
|CS038605000003|      20190910|
+--------------+--------------+
only showing top 10 rows



 > P-025: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上年月日（sales_ymd）を求め、10件表示せよ。

In [37]:
df_receipt.groupBy("customer_id") \
    .agg({"sales_ymd": "min"}) \
    .show(10)

+--------------+--------------+
|   customer_id|min(sales_ymd)|
+--------------+--------------+
|CS024415000195|      20170119|
|CS011515000044|      20170505|
|CS017415000245|      20180110|
|CS011512000113|      20181213|
|CS025414000093|      20170806|
|CS010605000007|      20170125|
|CS011415000097|      20170108|
|CS018515000090|      20170131|
|CS015415000222|      20170313|
|CS038605000003|      20170321|
+--------------+--------------+
only showing top 10 rows



 > P-026: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を求め、両者が異なるデータを10件表示せよ。

In [38]:
df_receipt.groupBy("customer_id") \
    .agg(F.min("sales_ymd"), F.max("sales_ymd")) \
    .filter(F.col("min(sales_ymd)") != F.col("max(sales_ymd)")) \
    .show(10)

+--------------+--------------+--------------+
|   customer_id|min(sales_ymd)|max(sales_ymd)|
+--------------+--------------+--------------+
|CS024415000195|      20170119|      20190907|
|CS011515000044|      20170505|      20181226|
|CS017415000245|      20180110|      20190615|
|CS011512000113|      20181213|      20190829|
|CS025414000093|      20170806|      20191016|
|CS010605000007|      20170125|      20181009|
|CS011415000097|      20170108|      20180909|
|CS018515000090|      20170131|      20190717|
|CS015415000222|      20170313|      20190928|
|CS038605000003|      20170321|      20190910|
+--------------+--------------+--------------+
only showing top 10 rows



 > P-027: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [39]:
df_receipt.groupBy("store_cd") \
    .agg(F.avg("amount")) \
    .orderBy(F.desc("avg(amount)")) \
    .show(5)

+--------+------------------+
|store_cd|       avg(amount)|
+--------+------------------+
|  S13052|402.86746987951807|
|  S13015|351.11196043165467|
|  S13003| 350.9155188246097|
|  S14010|348.79126213592235|
|  S13001| 348.4703862660944|
+--------+------------------+
only showing top 5 rows



 > P-028: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [40]:
df_receipt.groupBy("store_cd") \
    .agg(F.median("amount")) \
    .orderBy(F.desc("median(amount)")) \
    .show(5)

+--------+--------------+
|store_cd|median(amount)|
+--------+--------------+
|  S13052|         190.0|
|  S14010|         188.0|
|  S14050|         185.0|
|  S14040|         180.0|
|  S13003|         180.0|
+--------+--------------+
only showing top 5 rows



 > P-029: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求め、10件表示させよ。

In [41]:
df_receipt.groupBy("store_cd") \
    .agg(F.mode("product_cd")) \
    .show(10)

+--------+----------------+
|store_cd|mode(product_cd)|
+--------+----------------+
|  S14024|      P060303001|
|  S14045|      P060303001|
|  S13020|      P071401001|
|  S13038|      P060303001|
|  S14047|      P060303001|
|  S12014|      P060303001|
|  S13037|      P060303001|
|  S13051|      P071003001|
|  S12013|      P060303001|
|  S14046|      P060303001|
+--------+----------------+
only showing top 10 rows



 > P-030: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の分散を計算し、降順で5件表示せよ。

In [42]:
df_receipt.groupBy("store_cd") \
    .agg(F.variance("amount")) \
    .orderBy(F.desc("var_samp(amount)")) \
    .show(5)

+--------+------------------+
|store_cd|  var_samp(amount)|
+--------+------------------+
|  S13052| 441863.2525262341|
|  S14011| 306442.2424315683|
|  S14034| 297068.3927400608|
|  S13001|295558.84261771216|
|  S13015| 295427.1970858533|
+--------+------------------+
only showing top 5 rows



 > P-031: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標準偏差を計算し、降順で5件表示せよ。

In [43]:
df_receipt.groupBy("store_cd") \
    .agg(F.stddev("amount")) \
    .orderBy(F.desc("stddev(amount)")) \
    .show(5)

+--------+-----------------+
|store_cd|   stddev(amount)|
+--------+-----------------+
|  S13052|664.7279537722436|
|  S14011|  553.57225583619|
|  S14034|545.0398083994056|
|  S13001|543.6532374756101|
|  S13015|543.5321490821434|
+--------+-----------------+
only showing top 5 rows



 > P-032: レシート明細データ（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [44]:
window = Window.partitionBy("amount").orderBy(F.desc("amount"))
df_receipt.select("amount") \
    .withColumn("percentile", F.percentile("amount", [0.0, 0.25, 0.5, 0.75, 1.0]).over(window)) \
    .show(truncate=False)

+------+------------------------------+
|amount|percentile                    |
+------+------------------------------+
|10    |[10.0, 10.0, 10.0, 10.0, 10.0]|
|10    |[10.0, 10.0, 10.0, 10.0, 10.0]|
|10    |[10.0, 10.0, 10.0, 10.0, 10.0]|
|10    |[10.0, 10.0, 10.0, 10.0, 10.0]|
|10    |[10.0, 10.0, 10.0, 10.0, 10.0]|
|10    |[10.0, 10.0, 10.0, 10.0, 10.0]|
|10    |[10.0, 10.0, 10.0, 10.0, 10.0]|
|10    |[10.0, 10.0, 10.0, 10.0, 10.0]|
|10    |[10.0, 10.0, 10.0, 10.0, 10.0]|
|10    |[10.0, 10.0, 10.0, 10.0, 10.0]|
|10    |[10.0, 10.0, 10.0, 10.0, 10.0]|
|10    |[10.0, 10.0, 10.0, 10.0, 10.0]|
|10    |[10.0, 10.0, 10.0, 10.0, 10.0]|
|10    |[10.0, 10.0, 10.0, 10.0, 10.0]|
|10    |[10.0, 10.0, 10.0, 10.0, 10.0]|
|10    |[10.0, 10.0, 10.0, 10.0, 10.0]|
|10    |[10.0, 10.0, 10.0, 10.0, 10.0]|
|10    |[10.0, 10.0, 10.0, 10.0, 10.0]|
|10    |[10.0, 10.0, 10.0, 10.0, 10.0]|
|10    |[10.0, 10.0, 10.0, 10.0, 10.0]|
+------+------------------------------+
only showing top 20 rows



 > P-033: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [45]:
df_receipt.groupBy("store_cd") \
    .agg(F.avg("amount")) \
    .filter(F.col("avg(amount)") >= 330) \
    .show()

+--------+------------------+
|store_cd|       avg(amount)|
+--------+------------------+
|  S14045| 330.0820734341253|
|  S13020|  337.879932117098|
|  S14047| 330.0770731707317|
|  S12013|330.19412997903567|
|  S14011| 335.7183333333333|
|  S14010|348.79126213592235|
|  S13004|330.94394904458596|
|  S13052|402.86746987951807|
|  S13003| 350.9155188246097|
|  S13019|330.20861587554845|
|  S13001| 348.4703862660944|
|  S13015|351.11196043165467|
|  S14026|332.34058847239015|
+--------+------------------+



 > P-034: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [46]:
df_receipt.filter(F.col("customer_id").rlike("^(?!Z)")) \
    .groupBy("customer_id") \
    .agg(F.sum("amount")) \
    .agg(F.avg("sum(amount)"))\
    .show()

+-----------------+
| avg(sum(amount))|
+-----------------+
|2547.742234529256|
+-----------------+



 > P-035: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出し、10件表示せよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [47]:
avg = df_receipt.filter(F.col("customer_id").rlike("^(?!Z)")).groupBy("customer_id").agg(F.sum("amount")).agg(F.avg("sum(amount)")).first()[0]
df_receipt.filter(F.col("amount") >= avg).show(10)

+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
|sales_ymd|sales_epoch|store_cd|receipt_no|receipt_sub_no|   customer_id|product_cd|quantity|amount|
+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
| 20190615| 1560556800|  S13039|      1102|             2|CS039415000129|P090404061|       1|  2580|
| 20181025| 1540425600|  S14036|      1142|             2|ZZ000000000000|P040102023|       1|  5000|
| 20180907| 1536278400|  S14036|       112|             1|ZZ000000000000|P071301004|       1|  2680|
| 20181219| 1545177600|  S13015|      1132|             1|CS015515000083|P071301009|       1|  5280|
| 20190903| 1567468800|  S14011|      1192|             2|CS011415000006|P080104055|       1| 10925|
| 20181123| 1542931200|  S14045|      1142|             1|ZZ000000000000|P071301006|       1|  4760|
| 20181025| 1540425600|  S13004|      1142|             1|ZZ000000000000|P040102023|       

 > P-036: レシート明細データ（df_receipt）と店舗データ（df_store）を内部結合し、レシート明細データの全項目と店舗データの店舗名（store_name）を10件表示せよ。

In [48]:
df_receipt.join(df_store, on="store_cd", how="inner") \
    .select(*df_receipt.columns+["store_name"]) \
    .show(10)

+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+----------+
|sales_ymd|sales_epoch|store_cd|receipt_no|receipt_sub_no|   customer_id|product_cd|quantity|amount|store_name|
+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+----------+
| 20181103| 1541203200|  S14006|       112|             1|CS006214000001|P070305012|       1|   158|  葛が谷店|
| 20181118| 1542499200|  S13008|      1132|             2|CS008415000097|P070701017|       1|    81|    成城店|
| 20170712| 1499817600|  S14028|      1102|             1|CS028414000014|P060101005|       1|   170|  二ツ橋店|
| 20190205| 1549324800|  S14042|      1132|             1|ZZ000000000000|P050301001|       1|    25|  新山下店|
| 20180821| 1534809600|  S14025|      1102|             2|CS025415000050|P060102007|       1|    90|    大和店|
| 20190605| 1559692800|  S13003|      1112|             1|CS003515000195|P050102002|       1|   138|    狛江店|
| 20181205| 1

 > P-037: 商品データ（df_product）とカテゴリデータ（df_category）を内部結合し、商品データの全項目とカテゴリデータのカテゴリ小区分名（category_small_name）を10件表示せよ。

In [49]:
df_product.join(df_category, on=["category_major_cd", "category_medium_cd", "category_small_cd"], how="inner") \
    .select(*df_product.columns+["category_small_name"]) \
    .show(10)

+----------+-----------------+------------------+-----------------+----------+---------+-------------------+
|product_cd|category_major_cd|category_medium_cd|category_small_cd|unit_price|unit_cost|category_small_name|
+----------+-----------------+------------------+-----------------+----------+---------+-------------------+
|P040101001|                4|               401|            40101|       198|      149|             弁当類|
|P040101002|                4|               401|            40101|       218|      164|             弁当類|
|P040101003|                4|               401|            40101|       230|      173|             弁当類|
|P040101004|                4|               401|            40101|       248|      186|             弁当類|
|P040101005|                4|               401|            40101|       268|      201|             弁当類|
|P040101006|                4|               401|            40101|       298|      224|             弁当類|
|P040101007|                4|       

 > P-038: 顧客データ（df_customer）とレシート明細データ（df_receipt）から、顧客ごとの売上金額合計を求め、10件表示せよ。ただし、売上実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [50]:
df_customer.join(df_receipt, on="customer_id", how="inner") \
    .filter((F.col("gender_cd") == 1) & (F.col("customer_id").rlike("^(?!Z)"))) \
    .groupBy("customer_id") \
    .agg(F.sum("amount")) \
    .fillna(0) \
    .show(10)

+--------------+-----------+
|   customer_id|sum(amount)|
+--------------+-----------+
|CS024415000195|       4638|
|CS011515000044|       2533|
|CS017415000245|       1289|
|CS011512000113|        686|
|CS025414000093|       3929|
|CS011415000097|       3462|
|CS018515000090|       3756|
|CS015415000222|      11472|
|CS028515000242|       1525|
|CS040215000044|       1439|
+--------------+-----------+
only showing top 10 rows



 > P-039: レシート明細データ（df_receipt）から、売上日数の多い顧客の上位20件を抽出したデータと、売上金額合計の多い顧客の上位20件を抽出したデータをそれぞれ作成し、さらにその2つを完全外部結合せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [51]:
df_sale_date = df_receipt.filter(F.col("customer_id").rlike("^(?!Z)")) \
                .groupBy("customer_id").agg(F.count("sales_ymd")) \
                .orderBy(F.desc("count(sales_ymd)")) \
                .limit(20)
df_sale_amount = df_receipt.filter(F.col("customer_id").rlike("^(?!Z)")) \
                .groupBy("customer_id").agg(F.sum("amount")) \
                .orderBy(F.desc("sum(amount)")) \
                .limit(20)
df_sale_date.join(df_sale_amount, how="outer", on="customer_id").show()

+--------------+----------------+-----------+
|   customer_id|count(sales_ymd)|sum(amount)|
+--------------+----------------+-----------+
|CS001605000009|            NULL|      18925|
|CS006515000023|            NULL|      18372|
|CS007514000094|            NULL|      15735|
|CS007515000107|              40|       NULL|
|CS009414000059|            NULL|      15492|
|CS010214000002|              42|       NULL|
|CS010214000010|              44|      18585|
|CS011414000106|            NULL|      18338|
|CS011415000006|            NULL|      16094|
|CS014214000023|              38|       NULL|
|CS014415000077|              38|       NULL|
|CS015415000185|              44|      20153|
|CS015515000034|            NULL|      15300|
|CS016415000101|            NULL|      16348|
|CS016415000141|              40|      18372|
|CS017415000097|              40|      23086|
|CS021514000008|              36|       NULL|
|CS021514000045|              40|       NULL|
|CS021515000089|            NULL| 

 > P-040: 全ての店舗と全ての商品を組み合わせたデータを作成したい。店舗データ（df_store）と商品データ（df_product）を直積し、件数を計算せよ。

In [52]:
df_store.crossJoin(df_product).count()

531590

 > P-041: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を10件表示せよ。

In [53]:
window = Window.partitionBy().orderBy(F.asc("sales_ymd"))
df_receipt.groupBy("sales_ymd") \
    .agg(F.sum("amount")) \
    .withColumn("diff", F.col("sum(amount)") - F.lag("sum(amount)", 1).over(window)) \
    .show()

25/04/24 20:31:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/24 20:31:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/24 20:31:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---------+-----------+-----+
|sales_ymd|sum(amount)| diff|
+---------+-----------+-----+
| 20170101|      33723| NULL|
| 20170102|      24165|-9558|
| 20170103|      27503| 3338|
| 20170104|      36165| 8662|
| 20170105|      37830| 1665|
| 20170106|      32387|-5443|
| 20170107|      23415|-8972|
| 20170108|      24737| 1322|
| 20170109|      26718| 1981|
| 20170110|      20143|-6575|
| 20170111|      24287| 4144|
| 20170112|      23526| -761|
| 20170113|      28004| 4478|
| 20170114|      27452| -552|
| 20170115|      18793|-8659|
| 20170116|      34075|15282|
| 20170117|      36709| 2634|
| 20170118|      33976|-2733|
| 20170119|      26022|-7954|
| 20170120|      26326|  304|
+---------+-----------+-----+
only showing top 20 rows



25/04/24 20:31:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/24 20:31:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/24 20:31:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/24 20:31:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


> P-042: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、前回、前々回、3回前に売上があった日のデータを結合せよ。そして結果を10件表示せよ。

In [54]:
window = Window.orderBy(F.asc("sales_ymd"))
df_receipt.groupBy("sales_ymd") \
    .agg(F.sum("amount").alias("amount")) \
    .orderBy(F.asc("sales_ymd")) \
    .withColumn("diff_1", F.col("amount") - F.lag("amount", 1).over(window)) \
    .withColumn("diff_2", F.col("amount") - F.lag("amount", 2).over(window)) \
    .withColumn("diff_3", F.col("amount") - F.lag("amount", 3).over(window)) \
    .dropna() \
    .show(10)

25/04/24 20:31:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/24 20:31:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/24 20:31:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/24 20:31:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/24 20:31:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---------+------+------+------+------+
|sales_ymd|amount|diff_1|diff_2|diff_3|
+---------+------+------+------+------+
| 20170104| 36165|  8662| 12000|  2442|
| 20170105| 37830|  1665| 10327| 13665|
| 20170106| 32387| -5443| -3778|  4884|
| 20170107| 23415| -8972|-14415|-12750|
| 20170108| 24737|  1322| -7650|-13093|
| 20170109| 26718|  1981|  3303| -5669|
| 20170110| 20143| -6575| -4594| -3272|
| 20170111| 24287|  4144| -2431|  -450|
| 20170112| 23526|  -761|  3383| -3192|
| 20170113| 28004|  4478|  3717|  7861|
+---------+------+------+------+------+
only showing top 10 rows



25/04/24 20:31:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/24 20:31:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/24 20:31:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/24 20:31:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


> P-043： レシート明細データ（df_receipt）と顧客データ（df_customer）を結合し、性別コード（gender_cd）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータを作成せよ。性別コードは0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [55]:
df_sales_summary = df_receipt.join(df_customer, on="customer_id", how="inner") \
    .withColumn("age_group", F.floor(F.col("age")/10)*10) \
    .groupBy("age_group") \
    .pivot("gender") \
    .sum("amount")

df_sales_summary = df_sales_summary.selectExpr(
    "age_group",
    "`男性` as `0`",
    "`女性` as `1`",
    "`不明` as `9`"
)

df_sales_summary.show()

+---------+------+-------+------+
|age_group|     0|      1|     9|
+---------+------+-------+------+
|       50| 54320|6685192|342923|
|       10|  1591| 149836|  4317|
|       80| 46360| 262923|  5111|
|       30|177322| 693047| 50441|
|       90|  NULL|   6260|  NULL|
|       70| 13435|  29764|  2427|
|       20| 72940|1363724| 44328|
|       60|272469| 987741| 71418|
|       40| 19355|9320791|483512|
+---------+------+-------+------+



> P-044： 043で作成した売上サマリデータ（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする。

In [56]:
df_sales_summary.selectExpr(
    "age_group",
    "stack(3, " +
    "'99', `9`, " +
    "'01', `1`, " +
    "'00', `0`" +
    ") as (gender_code, amount)"
).show()

+---------+-----------+-------+
|age_group|gender_code| amount|
+---------+-----------+-------+
|       50|         99| 342923|
|       50|         01|6685192|
|       50|         00|  54320|
|       10|         99|   4317|
|       10|         01| 149836|
|       10|         00|   1591|
|       80|         99|   5111|
|       80|         01| 262923|
|       80|         00|  46360|
|       30|         99|  50441|
|       30|         01| 693047|
|       30|         00| 177322|
|       90|         99|   NULL|
|       90|         01|   6260|
|       90|         00|   NULL|
|       70|         99|   2427|
|       70|         01|  29764|
|       70|         00|  13435|
|       20|         99|  44328|
|       20|         01|1363724|
+---------+-----------+-------+
only showing top 20 rows



> P-045: 顧客データ（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [57]:
df_customer.select("birth_day", "customer_id") \
    .withColumn("birth_day", F.date_format("birth_day", format="yyyymmdd")) \
    .show(10)

+---------+--------------+
|birth_day|   customer_id|
+---------+--------------+
| 19810029|CS021313000114|
| 19520001|CS037613000071|
| 19760004|CS031415000172|
| 19330027|CS028811000001|
| 19950029|CS001215000145|
| 19740015|CS020401000016|
| 19770009|CS015414000103|
| 19730017|CS029403000008|
| 19310002|CS015804000004|
| 19620011|CS033513000180|
+---------+--------------+
only showing top 10 rows



> P-046: 顧客データ（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [58]:
df_customer.select("application_date", "customer_id") \
    .withColumn("application_date", F.to_date("application_date", format="yyyymmdd")) \
    .show(10)

+----------------+--------------+
|application_date|   customer_id|
+----------------+--------------+
|      2015-01-05|CS021313000114|
|      2015-01-14|CS037613000071|
|      2015-01-29|CS031415000172|
|      2016-01-15|CS028811000001|
|      2017-01-05|CS001215000145|
|      2015-01-25|CS020401000016|
|      2015-01-22|CS015414000103|
|      2015-01-15|CS029403000008|
|      2015-01-07|CS015804000004|
|      2015-01-28|CS033513000180|
+----------------+--------------+
only showing top 10 rows



> P-047: レシート明細データ（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型に変換し、レシート番号（receipt_no）、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [59]:
df_receipt.select("sales_ymd", "receipt_no", "receipt_sub_no") \
    .withColumn("sales_ymd", F.to_date("sales_ymd", format="yyyymmdd")) \
    .show(10)

+----------+----------+--------------+
| sales_ymd|receipt_no|receipt_sub_no|
+----------+----------+--------------+
|2018-01-03|       112|             1|
|2018-01-18|      1132|             2|
|2017-01-12|      1102|             1|
|2019-01-05|      1132|             1|
|2018-01-21|      1102|             2|
|2019-01-05|      1112|             1|
|2018-01-05|      1102|             2|
|2019-01-22|      1102|             1|
|2017-01-04|      1112|             2|
|2019-01-10|      1102|             1|
+----------+----------+--------------+
only showing top 10 rows



> レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [60]:
df_receipt.select("sales_epoch", "receipt_no", "receipt_sub_no") \
    .withColumn("sales_epoch", F.to_date(F.from_unixtime("sales_epoch", format="yyyymmdd"), format="yyyymmdd")) \
    .show(10)

+-----------+----------+--------------+
|sales_epoch|receipt_no|receipt_sub_no|
+-----------+----------+--------------+
| 2018-01-03|       112|             1|
| 2018-01-18|      1132|             2|
| 2017-01-12|      1102|             1|
| 2019-01-05|      1132|             1|
| 2018-01-21|      1102|             2|
| 2019-01-05|      1112|             1|
| 2018-01-05|      1102|             2|
| 2019-01-22|      1102|             1|
| 2017-01-04|      1112|             2|
| 2019-01-10|      1102|             1|
+-----------+----------+--------------+
only showing top 10 rows



> P-049: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「年」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。


In [61]:
df_receipt.select("sales_epoch", "receipt_no", "receipt_sub_no") \
    .withColumn("sales_epoch", F.year(F.to_date(F.from_unixtime("sales_epoch")))) \
    .show(10)

+-----------+----------+--------------+
|sales_epoch|receipt_no|receipt_sub_no|
+-----------+----------+--------------+
|       2018|       112|             1|
|       2018|      1132|             2|
|       2017|      1102|             1|
|       2019|      1132|             1|
|       2018|      1102|             2|
|       2019|      1112|             1|
|       2018|      1102|             2|
|       2019|      1102|             1|
|       2017|      1112|             2|
|       2019|      1102|             1|
+-----------+----------+--------------+
only showing top 10 rows



> P-050: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「月」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「月」は0埋め2桁で取り出すこと。

In [62]:
df_receipt.select("sales_epoch", "receipt_no", "receipt_sub_no") \
    .withColumn("sales_epoch", F.month(F.to_date(F.from_unixtime("sales_epoch")))) \
    .withColumn("sales_epoch", F.lpad("sales_epoch", 2, "0")) \
    .show(10)

+-----------+----------+--------------+
|sales_epoch|receipt_no|receipt_sub_no|
+-----------+----------+--------------+
|         11|       112|             1|
|         11|      1132|             2|
|         07|      1102|             1|
|         02|      1132|             1|
|         08|      1102|             2|
|         06|      1112|             1|
|         12|      1102|             2|
|         09|      1102|             1|
|         05|      1112|             2|
|         10|      1102|             1|
+-----------+----------+--------------+
only showing top 10 rows



> P-051: レシート明細データ（df_receipt）の売上エポック秒を日付型に変換し、「日」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「日」は0埋め2桁で取り出すこと。

In [ ]:
df_receipt.select("sales_epoch", "receipt_no", "receipt_sub_no") \
    .withColumn("sales_epoch", F.day(F.to_date(F.from_unixtime("sales_epoch")))) \
    .withColumn("sales_epoch", F.lpad("sales_epoch", 2, "0")) \
    .show(10)

+-----------+----------+--------------+
|sales_epoch|receipt_no|receipt_sub_no|
+-----------+----------+--------------+
|         03|       112|             1|
|         18|      1132|             2|
|         12|      1102|             1|
|         05|      1132|             1|
|         21|      1102|             2|
|         05|      1112|             1|
|         05|      1102|             2|
|         22|      1102|             1|
|         04|      1112|             2|
|         10|      1102|             1|
+-----------+----------+--------------+
only showing top 10 rows



> P-052: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に二値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
df_receipt.filter(F.col("customer_id").rlike("^(?!Z)")) \
    .groupBy("customer_id") \
    .agg(F.sum("amount").alias("amount")) \
    .withColumn("is_large", 
                F.when(F.col("amount") >= 2000, 1)
                .otherwise(0)) \
    .show(10)

+--------------+------+--------+
|   customer_id|amount|is_large|
+--------------+------+--------+
|CS024415000195|  4638|       1|
|CS011515000044|  2533|       1|
|CS017415000245|  1289|       0|
|CS011512000113|   686|       0|
|CS025414000093|  3929|       1|
|CS010605000007|  6420|       1|
|CS011415000097|  3462|       1|
|CS018515000090|  3756|       1|
|CS015415000222| 11472|       1|
|CS038605000003|  2833|       1|
+--------------+------+--------+
only showing top 10 rows



> P-053: 顧客データ（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に二値化せよ。さらにレシート明細データ（df_receipt）と結合し、全期間において売上実績のある顧客数を、作成した二値ごとにカウントせよ。

In [ ]:
df_customer.withColumn("is_tokyo", 
                       F.when(F.col("postal_cd")
                              .rlike("^1[0-9]{2}|^20[0-9]"), 1)
                              .otherwise(0)) \
    .join(df_receipt, on="customer_id", how="inner") \
    .select("customer_id", "is_tokyo") \
    .distinct() \
    .groupBy("is_tokyo") \
    .count() \
    .show()

+--------+-----+
|is_tokyo|count|
+--------+-----+
|       1| 4400|
|       0| 3906|
+--------+-----+



> P-054: 顧客データ（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに10件表示せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。

In [ ]:
df_customer.select("customer_id", "address") \
    .withColumn("address_code", 
                F.when(F.col("address").contains("埼玉県"), 11)
                .when(F.col("address").contains("千葉県"), 12)
                .when(F.col("address").contains("東京都"), 13)
                .when(F.col("address").contains("神奈川県"), 14)) \
    .show(10, truncate=False)

+--------------+----------------------------------+------------+
|customer_id   |address                           |address_code|
+--------------+----------------------------------+------------+
|CS021313000114|神奈川県伊勢原市粟窪**********    |14          |
|CS037613000071|東京都江東区南砂**********        |13          |
|CS031415000172|東京都渋谷区代々木**********      |13          |
|CS028811000001|神奈川県横浜市泉区和泉町**********|14          |
|CS001215000145|東京都大田区仲六郷**********      |13          |
|CS020401000016|東京都板橋区若木**********        |13          |
|CS015414000103|東京都江東区北砂**********        |13          |
|CS029403000008|千葉県浦安市海楽**********        |12          |
|CS015804000004|東京都江東区北砂**********        |13          |
|CS033513000180|神奈川県横浜市旭区善部町**********|14          |
+--------------+----------------------------------+------------+
only showing top 10 rows



> P-055: レシート明細（df_receipt）データの売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに10件表示せよ。カテゴリ値は順に1〜4とする。
>
> - 最小値以上第1四分位未満 ・・・ 1を付与
> - 第1四分位以上第2四分位未満 ・・・ 2を付与
> - 第2四分位以上第3四分位未満 ・・・ 3を付与
> - 第3四分位以上 ・・・ 4を付与

In [9]:
df_receipt_total = df_receipt.groupBy("customer_id") \
    .agg(F.sum("amount").alias("total_amount"))

percentile = df_receipt_total \
    .agg(F.percentile("total_amount", percentage=[0.25, 0.5, 0.75])) \
    .first()[0]
p1, p2, p3 = percentile

df_receipt_total.withColumn("category",
                            F.when(F.col("total_amount") < p1, 1)
                            .when((F.col("total_amount") >= p1) & (F.col("total_amount") < p2), 2)
                            .when((F.col("total_amount") >= p2) & (F.col("total_amount") < p3), 3)
                            .otherwise(4)
                            ) \
                            .show(10)


+--------------+------------+--------+
|   customer_id|total_amount|category|
+--------------+------------+--------+
|CS024415000195|        4638|       4|
|CS011515000044|        2533|       3|
|CS017415000245|        1289|       2|
|CS011512000113|         686|       2|
|CS025414000093|        3929|       4|
|CS010605000007|        6420|       4|
|CS011415000097|        3462|       3|
|CS018515000090|        3756|       4|
|CS015415000222|       11472|       4|
|CS038605000003|        2833|       3|
+--------------+------------+--------+
only showing top 10 rows



> P-056: 顧客データ（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに10件表示せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。

In [ ]:
df_customer.withColumn("age_group",
                       F.floor(F.col("age")/10)*10) \
    .withColumn("age_group",
                F.when(F.col("age_group") >= 60, 60)
                .otherwise(F.col("age_group"))) \
    .select("customer_id", "birth_day", "age_group") \
    .show(10)

+--------------+----------+---------+
|   customer_id| birth_day|age_group|
+--------------+----------+---------+
|CS021313000114|1981-04-29|       30|
|CS037613000071|1952-04-01|       60|
|CS031415000172|1976-10-04|       40|
|CS028811000001|1933-03-27|       60|
|CS001215000145|1995-03-29|       20|
|CS020401000016|1974-09-15|       40|
|CS015414000103|1977-08-09|       40|
|CS029403000008|1973-08-17|       40|
|CS015804000004|1931-05-02|       60|
|CS033513000180|1962-07-11|       50|
+--------------+----------+---------+
only showing top 10 rows



> P-057: 056の抽出結果と性別コード（gender_cd）により、新たに性別×年代の組み合わせを表すカテゴリデータを作成し、10件表示せよ。組み合わせを表すカテゴリの値は任意とする。

In [7]:
df_customer.withColumn("age_group",
                       F.floor(F.col("age")/10)*10) \
    .withColumn("age_group",
                F.when(F.col("age_group") >= 60, 60)
                .otherwise(F.col("age_group"))) \
    .select("customer_id", "birth_day", "age_group", "gender_cd") \
    .withColumn("gender_age_cd",
                F.concat_ws("_",F.col("gender_cd"), F.col("age_group"))) \
    .show(10)

+--------------+----------+---------+---------+-------------+
|   customer_id| birth_day|age_group|gender_cd|gender_age_cd|
+--------------+----------+---------+---------+-------------+
|CS021313000114|1981-04-29|       30|        1|         1_30|
|CS037613000071|1952-04-01|       60|        9|         9_60|
|CS031415000172|1976-10-04|       40|        1|         1_40|
|CS028811000001|1933-03-27|       60|        1|         1_60|
|CS001215000145|1995-03-29|       20|        1|         1_20|
|CS020401000016|1974-09-15|       40|        0|         0_40|
|CS015414000103|1977-08-09|       40|        1|         1_40|
|CS029403000008|1973-08-17|       40|        0|         0_40|
|CS015804000004|1931-05-02|       60|        0|         0_60|
|CS033513000180|1962-07-11|       50|        1|         1_50|
+--------------+----------+---------+---------+-------------+
only showing top 10 rows



> P-058: 顧客データ（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに10件表示せよ。

In [10]:
encoder = OneHotEncoder(inputCol="gender_cd", outputCol="dummy_gender_cd")
df_customer_dummy = encoder.fit(df_customer).transform(df_customer)

df_customer_dummy.select("customer_id", "dummy_gender_cd") \
    .show()

+--------------+---------------+
|   customer_id|dummy_gender_cd|
+--------------+---------------+
|CS021313000114|  (9,[1],[1.0])|
|CS037613000071|      (9,[],[])|
|CS031415000172|  (9,[1],[1.0])|
|CS028811000001|  (9,[1],[1.0])|
|CS001215000145|  (9,[1],[1.0])|
|CS020401000016|  (9,[0],[1.0])|
|CS015414000103|  (9,[1],[1.0])|
|CS029403000008|  (9,[0],[1.0])|
|CS015804000004|  (9,[0],[1.0])|
|CS033513000180|  (9,[1],[1.0])|
|CS007403000016|  (9,[0],[1.0])|
|CS035614000014|  (9,[1],[1.0])|
|CS011215000048|  (9,[1],[1.0])|
|CS009413000079|  (9,[1],[1.0])|
|CS040412000191|  (9,[1],[1.0])|
|CS029415000023|  (9,[1],[1.0])|
|CS009315000023|  (9,[1],[1.0])|
|CS040702000012|  (9,[0],[1.0])|
|CS046615000013|  (9,[1],[1.0])|
|CS025412000147|  (9,[1],[1.0])|
+--------------+---------------+
only showing top 20 rows



> P-059: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに10件表示せよ。標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
df_receipt_sum = df_receipt.filter(F.col("customer_id").rlike("^(?!Z)")) \
        .groupBy("customer_id") \
        .agg(F.sum("amount").alias("amount"))

stats = df_receipt_sum.agg(
            F.mean("amount").alias("mean"),
            F.stddev("amount").alias("stddev")
)

mean, stddev = stats.collect()[0]

df_receipt_sum.withColumn("z_score", (F.col("amount") - F.lit(mean)) / F.lit(stddev)) \
    .select("customer_id", "amount", "z_score") \
    .show(10)

+--------------+------+--------------------+
|   customer_id|amount|             z_score|
+--------------+------+--------------------+
|CS024415000195|  4638|  0.7682867318900646|
|CS011515000044|  2533|-0.00541859639243...|
|CS017415000245|  1289|-0.46265822982874355|
|CS011512000113|   686| -0.6842944830137067|
|CS025414000093|  3929|  0.5076895453094031|
|CS010605000007|  6420|  1.4232714801083135|
|CS011415000097|  3462|  0.3360409047830719|
|CS018515000090|  3756|  0.4441023615598201|
|CS015415000222| 11472|  3.2801642679863128|
|CS038605000003|  2833| 0.10484819623689982|
+--------------+------+--------------------+
only showing top 10 rows



> P-060: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。


In [11]:
df_receipt_sum = df_receipt.filter(F.col("customer_id").rlike("^(?!Z)")) \
        .groupBy("customer_id") \
        .agg(F.sum("amount").alias("amount"))

stats = df_receipt_sum.agg(
            F.min("amount").alias("min"),
            F.max("amount").alias("max")
)
_min, _max = stats.collect()[0]

df_receipt_sum.withColumn("normalized", (F.col("amount") - F.lit(_min)) / (F.lit(_max) - F.lit(_min))) \
    .select("customer_id", "amount", "normalized") \
    .show(10)

+--------------+------+--------------------+
|   customer_id|amount|          normalized|
+--------------+------+--------------------+
|CS024415000195|  4638| 0.19847062912756344|
|CS011515000044|  2533| 0.10701251303441084|
|CS017415000245|  1289|0.052963156065345844|
|CS011512000113|   686|  0.0267639902676399|
|CS025414000093|  3929|  0.1676659714980883|
|CS010605000007|  6420|  0.2758950295446646|
|CS011415000097|  3462| 0.14737573861661452|
|CS018515000090|  3756| 0.16014946124435175|
|CS015415000222| 11472|  0.4953945081682308|
|CS038605000003|  2833| 0.12004692387904067|
+--------------+------+--------------------+
only showing top 10 rows



> P-061: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底10）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。


In [ ]:
df_receipt_sum = df_receipt.filter(F.col("customer_id").rlike("^(?!Z)")) \
            .groupBy("customer_id") \
            .agg(F.sum("amount").alias("amount"))

df_receipt_sum.withColumn("amount_log", F.log10(F.col("amount"))) \
            .select("customer_id", "amount", "amount_log") \
            .show(10)

+--------------+------+------------------+
|   customer_id|amount|        amount_log|
+--------------+------+------------------+
|CS024415000195|  4638|3.6663307443019684|
|CS011515000044|  2533| 3.403635189790548|
|CS017415000245|  1289| 3.110252917353403|
|CS011512000113|   686|2.8363241157067516|
|CS025414000093|  3929| 3.594282028811806|
|CS010605000007|  6420| 3.807535028068853|
|CS011415000097|  3462| 3.539327063539375|
|CS018515000090|  3756|3.5747255835940734|
|CS015415000222| 11472| 4.059639138323725|
|CS038605000003|  2833| 3.452246574520437|
+--------------+------+------------------+
only showing top 10 rows



> P-062: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底e）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
df_receipt_sum = df_receipt.filter(F.col("customer_id").rlike("^(?!Z)")) \
            .groupBy("customer_id") \
            .agg(F.sum("amount").alias("amount"))

df_receipt_sum.withColumn("amount_log", F.log(F.col("amount"))) \
        .select("customer_id", "amount", "amount_log") \
        .show(10)

+--------------+------+-----------------+
|   customer_id|amount|       amount_log|
+--------------+------+-----------------+
|CS024415000195|  4638|8.442038517815478|
|CS011515000044|  2533|7.837159650001675|
|CS017415000245|  1289|7.161622002939187|
|CS011512000113|   686|6.530877627725885|
|CS025414000093|  3929| 8.27614021955846|
|CS010605000007|  6420|8.767173396684006|
|CS011415000097|  3462|8.149601735736155|
|CS018515000090|  3756|8.231109840328154|
|CS015415000222| 11472|9.347664562839402|
|CS038605000003|  2833|7.949091499830517|
+--------------+------+-----------------+
only showing top 10 rows



> P-063: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を10件表示せよ。

In [ ]:
df_product.withColumn("profit", F.col("unit_price") - F.col("unit_cost")) \
        .show(10)

+----------+-----------------+------------------+-----------------+----------+---------+------+
|product_cd|category_major_cd|category_medium_cd|category_small_cd|unit_price|unit_cost|profit|
+----------+-----------------+------------------+-----------------+----------+---------+------+
|P040101001|                4|               401|            40101|       198|      149|    49|
|P040101002|                4|               401|            40101|       218|      164|    54|
|P040101003|                4|               401|            40101|       230|      173|    57|
|P040101004|                4|               401|            40101|       248|      186|    62|
|P040101005|                4|               401|            40101|       268|      201|    67|
|P040101006|                4|               401|            40101|       298|      224|    74|
|P040101007|                4|               401|            40101|       338|      254|    84|
|P040101008|                4|          

> P-064: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。ただし、単価と原価には欠損が生じていることに注意せよ。

In [ ]:
profit_margin_avg = df_product.dropna() \
        .withColumn("profit", F.col("unit_price") - F.col("unit_cost")) \
        .withColumn("profit_margin", F.col("profit") / F.col("unit_price")) \
        .agg(F.mean("profit_margin").alias("profit_margin_avg")) \
        .first()[0]
print(profit_margin_avg)

0.24911389885176904


> P-065: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [44]:
df_product.dropna() \
        .withColumn("unit_price_new", F.floor(F.col("unit_cost") / F.lit(0.7))) \
        .withColumn("profit", F.col("unit_price_new") - F.col("unit_cost")) \
        .withColumn("profit_margin", F.col("profit") / F.col("unit_price_new")) \
        .show(10)

+----------+-----------------+------------------+-----------------+----------+---------+--------------+------+-------------------+
|product_cd|category_major_cd|category_medium_cd|category_small_cd|unit_price|unit_cost|unit_price_new|profit|      profit_margin|
+----------+-----------------+------------------+-----------------+----------+---------+--------------+------+-------------------+
|P040101001|                4|               401|            40101|       198|      149|           212|    63| 0.2971698113207547|
|P040101002|                4|               401|            40101|       218|      164|           234|    70|0.29914529914529914|
|P040101003|                4|               401|            40101|       230|      173|           247|    74|0.29959514170040485|
|P040101004|                4|               401|            40101|       248|      186|           265|    79| 0.2981132075471698|
|P040101005|                4|               401|            40101|       268|     

> P-066: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [43]:
df_product.dropna() \
        .withColumn("unit_price_new", F.round(F.col("unit_cost") / F.lit(0.7))) \
        .withColumn("profit", F.col("unit_price_new") - F.col("unit_cost")) \
        .withColumn("profit_margin", F.col("profit") / F.col("unit_price_new")) \
        .show(10)

+----------+-----------------+------------------+-----------------+----------+---------+--------------+------+-------------------+
|product_cd|category_major_cd|category_medium_cd|category_small_cd|unit_price|unit_cost|unit_price_new|profit|      profit_margin|
+----------+-----------------+------------------+-----------------+----------+---------+--------------+------+-------------------+
|P040101001|                4|               401|            40101|       198|      149|         213.0|  64.0| 0.3004694835680751|
|P040101002|                4|               401|            40101|       218|      164|         234.0|  70.0|0.29914529914529914|
|P040101003|                4|               401|            40101|       230|      173|         247.0|  74.0|0.29959514170040485|
|P040101004|                4|               401|            40101|       248|      186|         266.0|  80.0| 0.3007518796992481|
|P040101005|                4|               401|            40101|       268|     

> P-067: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [58]:
df_product.dropna() \
        .withColumn("unit_price_new", F.ceil(F.col("unit_cost") / F.lit(0.7))) \
        .withColumn("profit", F.col("unit_price_new") - F.col("unit_cost")) \
        .withColumn("profit_margin", F.col("profit") / F.col("unit_price_new")) \
        .show(10)

+----------+-----------------+------------------+-----------------+----------+---------+--------------+------+-------------------+
|product_cd|category_major_cd|category_medium_cd|category_small_cd|unit_price|unit_cost|unit_price_new|profit|      profit_margin|
+----------+-----------------+------------------+-----------------+----------+---------+--------------+------+-------------------+
|P040101001|                4|               401|            40101|       198|      149|           213|    64| 0.3004694835680751|
|P040101002|                4|               401|            40101|       218|      164|           235|    71| 0.3021276595744681|
|P040101003|                4|               401|            40101|       230|      173|           248|    75| 0.3024193548387097|
|P040101004|                4|               401|            40101|       248|      186|           266|    80| 0.3007518796992481|
|P040101005|                4|               401|            40101|       268|     

> P-068: 商品データ（df_product）の各商品について、消費税率10％の税込み金額を求めよ。1円未満の端数は切り捨てとし、結果を10件表示せよ。ただし、単価（unit_price）には欠損が生じていることに注意せよ。

In [55]:
df_product \
    .withColumn("unit_price_including_tax", F.when(F.col("unit_price").isNotNull(),
                                                   F.floor(F.col("unit_price") * 1.1))
                                                   .otherwise(0)) \
    .show(10)

+----------+-----------------+------------------+-----------------+----------+---------+------------------------+
|product_cd|category_major_cd|category_medium_cd|category_small_cd|unit_price|unit_cost|unit_price_including_tax|
+----------+-----------------+------------------+-----------------+----------+---------+------------------------+
|P040101001|                4|               401|            40101|       198|      149|                     217|
|P040101002|                4|               401|            40101|       218|      164|                     239|
|P040101003|                4|               401|            40101|       230|      173|                     253|
|P040101004|                4|               401|            40101|       248|      186|                     272|
|P040101005|                4|               401|            40101|       268|      201|                     294|
|P040101006|                4|               401|            40101|       298|      224|

> P-069: レシート明細データ（df_receipt）と商品データ（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。

In [16]:
df_receipt_joined = df_receipt.join(df_product, on="product_cd", how="inner")

df_receipt_all = df_receipt_joined \
    .groupBy(F.col("customer_id")) \
    .agg(F.sum("amount").alias("amount_all")) \

df_receipt_07 = df_receipt_joined \
    .filter(F.col("category_major_cd") == "07") \
    .groupBy(F.col("customer_id")) \
    .agg(F.sum("amount").alias("amount_07"))

df_receipt_all.join(df_receipt_07, on="customer_id", how="inner") \
    .withColumn("ratio", F.col("amount_07") / F.col("amount_all")) \
    .show(10)

+--------------+----------+---------+-------------------+
|   customer_id|amount_all|amount_07|              ratio|
+--------------+----------+---------+-------------------+
|CS024415000195|      4638|     1431|0.30853816300129366|
|CS011515000044|      2533|     1064|0.42005527043031976|
|CS017415000245|      1289|      607| 0.4709076803723817|
|CS025414000093|      3929|     1010| 0.2570628658691779|
|CS010605000007|      6420|     3946| 0.6146417445482866|
|CS011415000097|      3462|     1436|  0.414789139225881|
|CS018515000090|      3756|     1594|0.42438764643237487|
|CS015415000222|     11472|     9314| 0.8118898186889819|
|CS038605000003|      2833|     1673| 0.5905400635368867|
|CS028515000242|      1525|      438| 0.2872131147540984|
+--------------+----------+---------+-------------------+
only showing top 10 rows



> P-070: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [9]:
df_receipt.join(df_customer, on="customer_id", how="inner") \
    .withColumn("sales_ymd", F.to_date("sales_ymd",
                                       "yyyymmdd")) \
    .withColumn("application_date", F.to_date("application_date",
                                              "yyyymmdd")) \
    .withColumn("elapsed_days", F.date_diff(end="sales_ymd",
                                            start="application_date")) \
    .select("customer_id", "sales_ymd", "application_date", "elapsed_days") \
    .show(10)

+--------------+----------+----------------+------------+
|   customer_id| sales_ymd|application_date|elapsed_days|
+--------------+----------+----------------+------------+
|CS006214000001|2018-01-03|      2015-01-01|        1098|
|CS008415000097|2018-01-18|      2015-01-22|        1092|
|CS028414000014|2017-01-12|      2015-01-11|         732|
|CS025415000050|2018-01-21|      2016-01-31|         721|
|CS003515000195|2019-01-05|      2015-01-06|        1460|
|CS024514000042|2018-01-05|      2015-01-10|        1091|
|CS040415000178|2019-01-22|      2015-01-27|        1456|
|CS027514000015|2019-01-10|      2015-01-01|        1470|
|CS025415000134|2019-01-18|      2015-01-20|        1459|
|CS021515000126|2017-01-10|      2015-01-08|         733|
+--------------+----------+----------------+------------+
only showing top 10 rows



> P-071: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [13]:
df_receipt.join(df_customer, on="customer_id", how="inner") \
    .withColumn("sales_ymd", F.to_date("sales_ymd",
                                       "yyyymmdd")) \
    .withColumn("application_date", F.to_date("application_date",
                                              "yyyymmdd")) \
    .withColumn("elapsed_months", F.floor(F.months_between("sales_ymd",
                                            "application_date"))) \
    .select("customer_id", "sales_ymd", "application_date", "elapsed_months") \
    .show(10)

+--------------+----------+----------------+--------------+
|   customer_id| sales_ymd|application_date|elapsed_months|
+--------------+----------+----------------+--------------+
|CS006214000001|2018-01-03|      2015-01-01|            36|
|CS008415000097|2018-01-18|      2015-01-22|            35|
|CS028414000014|2017-01-12|      2015-01-11|            24|
|CS025415000050|2018-01-21|      2016-01-31|            23|
|CS003515000195|2019-01-05|      2015-01-06|            47|
|CS024514000042|2018-01-05|      2015-01-10|            35|
|CS040415000178|2019-01-22|      2015-01-27|            47|
|CS027514000015|2019-01-10|      2015-01-01|            48|
|CS025415000134|2019-01-18|      2015-01-20|            47|
|CS021515000126|2017-01-10|      2015-01-08|            24|
+--------------+----------+----------------+--------------+
only showing top 10 rows



> P-072: レシート明細データ（df_receipt）の売上日（df_customer）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [14]:
df_receipt.join(df_customer, on="customer_id", how="inner") \
    .withColumn("sales_ymd", F.to_date("sales_ymd",
                                       "yyyymmdd")) \
    .withColumn("application_date", F.to_date("application_date",
                                              "yyyymmdd")) \
    .withColumn("elapsed_years", F.floor(F.months_between("sales_ymd",
                                            "application_date") / 12)) \
    .select("customer_id", "sales_ymd", "application_date", "elapsed_years") \
    .show(10)

+--------------+----------+----------------+-------------+
|   customer_id| sales_ymd|application_date|elapsed_years|
+--------------+----------+----------------+-------------+
|CS006214000001|2018-01-03|      2015-01-01|            3|
|CS008415000097|2018-01-18|      2015-01-22|            2|
|CS028414000014|2017-01-12|      2015-01-11|            2|
|CS025415000050|2018-01-21|      2016-01-31|            1|
|CS003515000195|2019-01-05|      2015-01-06|            3|
|CS024514000042|2018-01-05|      2015-01-10|            2|
|CS040415000178|2019-01-22|      2015-01-27|            3|
|CS027514000015|2019-01-10|      2015-01-01|            4|
|CS025415000134|2019-01-18|      2015-01-20|            3|
|CS021515000126|2017-01-10|      2015-01-08|            2|
+--------------+----------+----------------+-------------+
only showing top 10 rows

